In [ ]:
import cvxpy as cp  # note: also need cvxopt installed
import numpy as np
import pandas as pd

# Setup

In [ ]:
# Define players and their point projections
players = pd.DataFrame(
    [
        ["QB1", "QB", 20, 15],
        ["QB2", "QB", 18, 16],
        ["QB3", "QB", 17, 16],
        ["RB1", "RB", 16, 14],
        ["RB2", "RB", 12, 16],
        ["RB3", "RB", 13, 14],
    ],
    columns=["uid", "position", "week1", "week2"],
)
all_idx = set(players.index)
week_cols = [c for c in players.columns if c.startswith("week")]

players

In [ ]:
# Define draft order
draft = [1, 2, 3, 3, 2, 1]

In [ ]:
# Define constraints
num_players = 2
pos_const = {"QB": 1, "RB": 1}

In [ ]:
def display_draft(picks_idx):
    for team, idx in picks_idx.items():
        print(f"--- Team {team} ---")
        roster = players.loc[idx, ["uid"] + week_cols].set_index("uid")
        sum_points = roster.sum(axis=0).to_frame().rename({0: "sum_points"}, axis=1).T
        sum_points["min"] = sum_points.min(axis=1)
        display(roster)
        display(sum_points.sort_index(axis=1))

# Method 1: Naive
Optimize as if you get all picks in a row.

In [ ]:
# Keep track of picks
picks_idx = {
    1: [],
    2: [],
    3: [],
}
picked_idx = []

# Optimize draft
for team in draft:
    # Get available players and those already picked by the team
    available_idx = all_idx - set(picked_idx)
    prev_picks_idx = set(picks_idx[team])
    available_idx |= prev_picks_idx
    available_idx = sorted(available_idx)

    # Get data
    uid_vals = players.loc[available_idx, "uid"].values
    pos_vals = players.loc[available_idx, "position"].values
    points_vals = players.loc[available_idx, week_cols].values

    # The variable we are solving for. We define our output variable as a bool
    # since we have to make a binary decision on each player (pick or don't pick)
    roster = cp.Variable(len(available_idx), boolean=True)

    # Save constraints
    constraints = []

    # Our roster must be composed of exactly `num_players` players
    constraints.append(cp.sum(roster) == num_players)

    # Define position constraints
    for pos, num in pos_const.items():
        is_pos = pos_vals == pos
        constraints.append(cp.sum(is_pos @ roster) == num)

    # Define constraints corresponding already picked players
    for idx in prev_picks_idx:
        did_pick = uid_vals == players.loc[idx, "uid"]
        constraints.append(cp.sum(did_pick @ roster) == 1)

    # Define the objective
    weekly_points = roster @ points_vals
    min_weekly_points = cp.min(weekly_points)
    objective = cp.Maximize(min_weekly_points)

    # Solve
    problem = cp.Problem(objective, constraints)
    problem.solve(max_iters=25)

    # Get result
    roster_idx = roster.value
    opt_roster = np.array(available_idx)[roster_idx.astype(bool)]  # re-align indices
    new_pick_idx = list(set(opt_roster) - prev_picks_idx)[0]  # TODO: use ADP or something instead of first
    picks_idx[team].append(new_pick_idx)
    picked_idx.append(new_pick_idx)

# Display draft
display_draft(picks_idx)